# Using Reinforcement Learning to Play a Fishing Game
*By Keegan Kochis and Landon Zweigle*

# Introduction
---
Ever since we have gotten into programming we have had a desire to do reinforcement learning, and this term project seemed like a perfect excuse to finally do it. When it came to deciding what to do RL on, we decided to use a [JavaFX](https://openjfx.io/) game Landon made a few years ago; a very simple fishing minigame where the goal is to put a bobber over a fish for a certain amount of time. This game had everything we were looking for; an evironment which is rather simple, but not too simple; and an easy way to communicate with it.

The game as described earlier is a simple fishing minigame. It is actually a clone of minigame in a very popular indie game [Stardew Valley](https://www.stardewvalley.net/). Here is an indepth [description](https://stardewvalleywiki.com/Fishing#Overview_.26_Controls) of the fishing minigame. In short, the goal is to keep a capture area (aka "the bobber") behind the goal (aka "the fish") for a certain amount of time. The challenge arrises with sporadic movement of the fish, and the clunky/slow movement of the bobber.

Because the game was built in Java and our RL algorithm would be implemented in Python, we needed a way to communicate between Python and Java. For this we decided to use socket programming. Further, because of the random movement of the fish, we knew we had to use a neural network to approximate the Q-table.

# Methods
---
For our solution we of course needed a nueral network class. For this, we used some code provided to us in the course, a nueral network class and it's transitive optimizer class. We regularly visited lectures (lectures [17](https://nbviewer.jupyter.org/url/www.cs.colostate.edu/~cs445/notebooks/17%20Reinforcement%20Learning%20with%20Neural%20Network%20as%20Q%20Function.ipynb), [18](https://nbviewer.jupyter.org/url/www.cs.colostate.edu/~cs445/notebooks/18.1%20Reinforcement%20Learning%20to%20Control%20a%20Marble.ipynb)) for information regarding reinforcement learning algorithms, and a bit of code. Everything else was written by us.

## Steps taken
#### Java-Python Communications
Because Landon wrote the game, he implemented the communication between Java and Python. There were several options he could have gone with but he ended up writting a simple java socket server which python would recieve the games state from, and map actions to, all while keeping the game loop synced to the RL algorithm loops.

#### Convenient RL Neural Network
Keegan integrated the neural network class with a few modifications. He implemented a trivial `EpsilonGreedy` method which had access to the nueral networks `self` variable. He also implemented a template `getReinforcement` method which would be later greatly tuned to increase algorithm performance.

#### Main loop
Landon worked on the "main loop" of our program. The main loop deals with the main processes in the RL algorithm; getting the data for each trial, training the model, saving the results, etc..

#### Secondary Code
For this project to work properly a lot of secondary code was neccessary. Such functions that were neccesary include a way of easily creating a csv of experiments to be run, and a way of executing experiments. At this point in development, we both contributed to this code.

#### Fine tunning
To experiment with producing a functional model both Landon and Keegan would frequently adjust what Java reports as the state, and what the reinforcement/reward is defined as.


## Fine tunning
Because the bobber and the fish both have a veloctity, and thus a position, the reinforcement is a little more complex than just how far the bobber is from the fish. Throughout development we kept an ideaology in mind such that there are technically 4 types of states:
* `Best Colliding`: The bobber is behind the fish and their velocities are in the same direction (equal normalized velocity).

* `Simple Colliding`: The bobber is behind the fish, but they have differing normalized velocities.

* `Seperate`: The bobber is not behind the fish, but they have equivalent normalized velocities.

* `Poor Seperate`: The bobber is not behind the fish and their normalized velocites are inequivalant.

In general we knew we wanted to minimize the reinforcement, and as such we decided that the reinforcement each state was as such: `Best Colliding` < `Simple Colliding` < `Seperate` < `Poor Seperate`.

We first began with a simple reinforcement based off of this ideaoligy that follows the above definition. `Best Colliding` returned 0, `Simple Colliding` returned 1, `Seperate` returned 2, `Poor Seperate` returned 3.

After discussing our problem we decided that our reinforcement should also encode the fish-bobber distance. To do this we realized that we couldn't just return the distance between the two. For instance, we considered an example where the fish and bobber are not colliding and moving in the same direction. If the fish's position is greater than the bobber's position, and their velocities are upward, then the subsequent state is `Seperate`, but if their velocites are downward, then the subsequent state is `Poor Seperate`. However, if we just return the distance this ideaology would be lost. Because of this we only encode their distance when the state is `Seperate`. This introduced another problem, `Seperate`'s reinforcement value according to the ideaology should be between 1 and 3, whereas the distance could be anywhere from 0 to 1000. This meant we needed to have a standardized distance between 0 and 1, so we simply take the percentage of the distance over the total possible distance (distance/max_range). This didn't entirly solve our problem though, as the new reinforcement for `Seperate` would be between 0 and 1, when it should be between 1 and 3, so we simply added 1 to the percentage. This gave us a reinforcement function defined as:

    if `Best Colliding`:
        return 0
    elif `Simple Colliding`:
        return 1
    elif `Seperate`:
        return (distance/max_range) + 1
    elif `Poor Seperate`:
        return 3

